# Model design

Two qubits that evolve with the same Lindbladian (or different ones, it can come in a jnp.array where the first axis is the number of components)

There should be a way of check errors in jax, but I do not know. Probably try catch before calling something with jit

I guess that the main loop can be done with jit, but before we could check that everything is correct. 

For now let's focus on writing things modular.

1. Write everything in row based vectorization to make it compatible with flat and reshape
2. Work as much as we can with vectorized version
3. 